In [1]:
import sys
sys.path.append('/usr/lib/python2.7/dist-packages')

In [2]:
import rospy, cv2
from sensor_msgs.msg import Image
from cv_bridge import CvBridge, CvBridgeError

#### DaHua NetCam

In [3]:
class DaHuaNetCam:
    '''
    DaHua NetCam
    '''
    def __init__(self, user, pwd, ip, channel):
        # Default parameters, can be set.
        self._fps = 30
        self._netCamId = 1
        self._img_size = (1280, 720)
        # Get the value from config.
        self.user = user
        self.pwd = pwd
        self.ip = ip
        self.channel = channel
    
    def openNetCam(self):
        import cv2
        # NetCam URL:
        url = 'rtsp://%s:%s@%s/cam/realmonitor?channel=%d&subtype=0' \
              % (self.user, self.pwd, self.ip, self.channel)
        netCam = cv2.VideoCapture(url)
        #netCam = cv2.VideoCapture(0)
        # Check the camera death or life.
        if netCam.isOpened():
            return netCam
        raise Exception("NetCam can't open! [NetCam URL]: %s" % url)
    
    def set_fps(self, fps):
        self._fps = fps
        
    def set_netCamId(self, netCamId):
        self._netCamId = netCamId
        
    def set_img_size(self, img_size):
        self._img_size = img_size

#### HiKiViSon NetCam

In [4]:
class HiKiViSonNetCam:
    
    def __init__(self, user, pwd, ip, channel):
        # Default parameters, can be set.
        self._fps = 30
        self._netCamId = 1
        self._img_size = (1280, 720)
        # Get the value from config.
        self.user = user
        self.pwd = pwd
        self.ip = ip
        self.channel = channel
    
    def openNetCam(self):
        import cv2
        # NetCam URL:
        # HiKiViSon new version 2017
        url = "rtsp://%s:%s@%s//Streaming/Channels/%d" \
              % (self.user, self.pwd, self.ip, self.channel)
        netCam = cv2.VideoCapture(url)
        #netCam = cv2.VideoCapture(0)
        if not netCam.isOpened():
            # HiKiViSon old version 2015
            url = 'rtsp://%s:%s@%s/h265/ch%s/main/av_stream' \
            % (self.user, self.pwd, self.ip, self.channel)
            netCam = cv2.VideoCapture(url)
            # Check the camera death or life.
            if netCam.isOpened():
                return netCam
            else:
                raise Exception("NetCam can't open! [NetCam URL]: %s" % url)
        return netCam
        
    def set_fps(self, fps):
        self._fps = fps
        
    def set_netCamId(self, netCamId):
        self._netCamId = netCamId
        
    def set_img_size(self, img_size):
        self._img_size = img_size

#### NetCam Converter -> CV2 -> sensor_msgs.msg.Image

In [5]:
class NetCam_Converter:
    '''
    This class can convert the NetCam opencv image to ros messages.
    Also it will get a frame_id and append to the message and ros time stamp.
    '''
    def __init__(self, image_topic, frame_id):
        self.image_pub = rospy.Publisher(image_topic, Image)
        self._frame_id = frame_id
        self.bridge = CvBridge()
    
    def publish(self, image):
        try:
            # opencv img -> ros msg.Image
            image_msg = self.bridge.cv2_to_imgmsg(image, 'bgr8')
            # ROS Time Stamp
            image_msg.header.stamp = rospy.Time.now()
            # Frame id from the camera param.
            image_msg.header.frame_id = self._frame_id
            self.image_pub.publish(image_msg)
        except CvBridgeError as e:
            print(e)
            
def main():
    rospy.init_node('netCam', anonymous=False)
    # Get parameters
    user, pwd, ip, channel = [rospy.get_param('~user', 'admin'),
                              rospy.get_param('~pwd', 'admin2019'),
                              rospy.get_param('~ip', '172.16.1.180'),
                              rospy.get_param('~channel', 1)]
    camera = rospy.get_param('~camera', 'netCam')
    output_topic = rospy.get_param('~output_topic', '/netCam/image')
    netCam_type = rospy.get_param('~netCam_type', 'DaHua')
    if netCam_type == 'Dahua':
        netCam = DaHuaNetCam(user, pwd, ip, channel)
    else:
        netCam = HiKiViSonNetCam(user, pwd, ip, channel)
    img_pub = NetCam_Converter(output_topic, camera)
    while netCam.isOpened() and not rospy.is_shutdown():
        try:
            ret, frame = netCam.read()
            if ret:
                img_pub.publish(frame)
        except Exception as e:
            print(e)

#### Main Function

In [6]:
user, pwd, ip, channel = ['admin', 'admin2019', '172.16.1.180', 1]

In [7]:
netCam_type = 'Dahua'
if netCam_type == 'Dahua':
    netCam = DaHuaNetCam(user, pwd, ip, channel)
else:
    netCam = HiKiViSonNetCam(user, pwd, ip, channel)
    
netCam = netCam.openNetCam()
netCam.isOpened()

True

In [8]:
import time
while netCam.isOpened():
    t0 = time.time()
    ret, frame = netCam.read()
    if ret:
        t1 = time.time()
        cv2.imshow('HiKiViSon', frame)
        print(1./(t1-t0))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
cv2.destroyAllWindows()

31.1360339695
350.255031315
406.819010669
416.142871317
394.943879473
245.151908352
340.143054091
263.015237976
357.387866394
377.219534131
416.30808933
440.717032678
382.691970803
442.67060686
411.165964121
280.12449075
394.646593903
378.957715938
427.510345531
402.099894545
391.515355176
187.547129315
374.257517623
129.649902631
326.176530057
412.743948042
154.657227139
406.346056966
403.570095256
399.039482447
335.893649395
93.101240816
301.857070889
390.749394448
261.441376301
302.663010535
400.79350215
381.404383014
385.825039095
409.640003907
287.675171468
399.039482447
388.649369904
392.761869089
207.300153215
296.983926928
400.946754612
364.817256676
441.087811547
206.402440825
388.50537236
380.228809718
277.236036751
288.526105799
374.224125625
293.595408092
398.736001521
277.694915254
356.385759198
261.311070961
307.997062711
266.237400025
285.540472462
258.588409371
398.092634776
310.459215396
489.017605223
395.726389282
278.561732085
495.780614657
310.942545778
491.42401874

35.8564137636
52.815674818
36.5297032721
61.6374324007
21.1586684222
48.4336308733
49.5388285873
22.3174876821
53.7317960543
49.4436402216
19.2455766831
61.66371161
68.23112962
20.5562830817
40.9432068878
59.56633624
19.4799409235
31.881543642
72.5419671734
16.5256949233
80.3275687063
78.0219502214
40.264029951
35.2672939317
73.642419454
16.8483835722
51.0206305956
38.5775357787
37.1249623821
76.4700176849
41.6147000169
43.0737252888
16.2454063768
67.6631606117
29.6937671679
120.194406236
50.0704803744
18.4498823322
34.7717203873
58.6821126268
30.0843793484
64.1115221179
38.8483781931
17.376784575
33.2604099758
66.8532172971
201.649230769
37.451817987
31.8017727028
18.925400342
38.6397295231
49.6179435007
44.6050706143
19.0171295918
46.2921913802
60.0455820878
23.468051297
68.9081947822
20.9955599161
44.4406018224
66.203204167
21.9336390782
54.609067
45.3310853166
16.9497646037
39.6088882174
75.9479955094
34.800859587
57.3556503665
41.2200404898
17.6379478553
36.1808740058
69.208369084

57.3125452632
25.1674347158
167.390509638
47.865429605
17.5146529477
34.8239748263
80.353730028
22.294000085
166.03214314
35.9104444387
21.8875123937
38.3363556595
51.6317350896
20.3720717876
64.4702265671
43.320188802
62.4961482872
20.6637336867
47.8998675255
49.0217858813
16.9422333529
75.3476808105
79.2514549165
47.384163494
32.798748827
77.1068460916
24.599590623
43.4291867713
35.9671054324
20.0734345387
39.2324687351
85.208516171
36.2531137906
48.028764786
43.8945936329
42.4520399591
16.9170864709
81.2881119423
22.9143097524
37.5884213828
122.279350456
27.174685447
41.9816631301
33.767844779
14.5057842551
78.2285884811
78.6097908389
61.3157517725
55.9755508401
45.4071515952
16.4736397664
55.1294541344
77.5315908167
48.9046114382
55.4970956773
43.5464191532
19.9900104852
53.838700982
24.1487742248
43.8982688967
14.8480398751
24.7924008583
103.950630747
211.82283723
210.219727346
20.6028323157
44.5496394014
82.149440821
19.6324862737
58.6066762614
27.119338424
56.2141181832
77.82650

27.6342840577
55.9315108681
44.6787178968
17.2834125879
28.9922167692
74.1724552592
36.1362982364
139.531071191
43.8079441839
42.611183355
17.3067327966
55.8748834359
36.4563581052
53.6767852572
64.2923449523
20.8403301219
41.9900688771
57.1789404804
26.1192280627
121.313819633
81.8528550798
17.4245737645
39.7534215415
47.730344239
47.6777156368
30.7173752243
43.7983375799
42.3316444965
16.2863455453
88.9132342654
78.4568649458
66.9439141954
37.8728453141
40.604315711
37.2148884255
16.8033620313
71.1501950806
83.870983223
33.814669698
47.0039559356
17.5746849021
77.4113912369
67.0873960333
49.5265444927
46.6308381602
39.3023173006
14.695045599
77.609059285
96.6652224015
28.7290934621
28.4245894862
103.358896008
21.6469033856
47.6712129478
76.2143441208
171.55319236
25.619233189
16.9877279244
75.4140640452
75.5853021211
55.1084482985
44.0393112138
41.8551441972
15.8896819263
77.0360357051
77.6651050829
172.953857573
24.5959842372
43.3313773296
16.0875121779
44.2497810881
81.2550417482
4

36.3438989307
24.4552997219
108.827067279
48.5648584496
76.3878487652
22.3318655926
41.1488668694
19.8831181144
67.1862625745
72.9228575899
23.5809926462
59.4742708053
78.4187263957
27.993939758
22.6752228704
78.5112030399
36.3265864664
80.1097083485
87.5199065187
25.7353998699
42.3513065956
35.5100409767
22.2435154299
43.9520900355
28.8575733599
94.5984031756
37.9850027169
20.8572224211
32.1294276259
61.66371161
16.3805447288
58.4588281206
58.4213722595
29.6366976626
73.8056977951
56.983180719
21.8039965898
46.3802373028
127.146356251
19.6456360247
72.0460346634
44.1227014517
19.5771383229
74.2551827919
77.2616648553
68.2366798445
24.5519276022
41.9304608617
16.3596522363
77.0657602205
65.2830282655
31.879847377
54.0426485936
39.9043278882
16.5664902441
44.7707612826
76.051277402
26.7529707423
70.5067240452
43.9345952004
70.1811123753
20.1955076197
75.2395508198
22.7396407679
77.6464141582
102.07106006
59.844250717
19.4148382677
65.4490754467
18.7836056177
75.4439068262
62.5698003998


...

17.747132273
36.1793135572
38.340560898
22.0176905673
56.9120464599
78.6274745051
20.4629142659
74.8675365475
38.8861961228
16.0924500648
70.2798927614
94.1714901547
64.8931522109
29.564834917
53.3192311604
17.7651727892
78.0524405903
70.7219046656
37.6491539877
31.3822763595
49.4587991132
16.7205000638
80.3214155767
87.2976730633
19.58326252
79.6291078921
96.6741345134
18.045993727
31.185575672
78.2957625537
45.7898448673
59.2482766414
41.2151799226
21.0485577213
158.754882665
44.1403464461
17.8208778929
73.0142571155
72.616066482
22.3239019821
75.3910198799
9.61630205014
91.9541358823
98.0504476705
14.8200766744
335.571165693
420.186736125
481.274125072
48.3231448092
25.487527573
20.3280360198
79.0513023484
88.6201694521
27.019061423
65.3684932361
64.9222815572
29.4332991818
78.6230528427
175.501234361
19.5939662059
35.488708572
21.433957636
77.0119897912
75.9136305225
137.893414867
28.2304037046
39.8376216935
42.0574362265
21.7060528277
77.6449767674
27.3440511115
46.3151943463
45.8

84.817374775
14.4573343077
418.259274033
511.562873521
337.950527758
408.165044764
451.437304919
22.1047183882
33.2293163686
57.0739022167
22.4981306556
64.9222815572
81.6393646839
21.3990734883
70.1306536024
25.7281381882
102.080996885
42.1178289903
29.6612190344
18.2741622771
75.1761690535
75.4955091168
41.6928827038
34.7971062587
89.8137901499
30.412680458
78.6348450477
183.960701754
19.3839726407
44.2082718496
62.5436760013
20.0581711755
77.8872072942
89.5663798074
29.6595410671
48.6267926497
45.1792281095
17.4070013073
79.5038289484
113.777777778
24.2806018189
38.1678572404
61.2629118953
18.2378488377
84.0541883768
82.8308154117
32.0195431782
45.6726702528
5.93676123182
86.8511792599
88.1749075008
19.1430696978
348.19060269
464.69133614
406.661237153
397.112668055
28.7084462697
40.3764343473
33.6905417888
17.1066210958
58.8756878158
87.3212999396
24.5345500278
79.2394770649
71.3668986405
27.7230539417
60.9974113609
76.7288160398
31.7247993707
37.3092332325
50.0024319878
16.6778162

31.6534522704
51.725335438
36.7621501757
17.4898212782
28.8250486224
97.1645933236
25.7644876347
7.08797817994
84.7745169375
84.0980069776
13.8266616999
350.021196695
378.923479989
356.385759198
419.472347235
406.18865001
28.4373088892
38.2919066965
36.0645227859
15.8586811857
76.1783541292
69.9890535309
25.0407703927
135.208536153
46.9971091141
17.3343417436
73.0778639254
63.6165688371
26.5285980836
88.9924678025
127.906318614
19.1175915586
57.3062808269
44.4029642177
21.0859110675
67.6129864268
85.9330041591
26.1431598893
39.1497083119
62.4096658036
23.6294823186
89.977560871
39.1862848601
28.2351547301
68.8516366263
81.1293061761
23.3797512807
59.9631726425
74.6224491611
19.4924341005
34.5733786145
96.2923917535
24.1714585386
129.987417485
16.7599867336
50.4741871044
91.1171359055
10.8704939574
176.23866549
172.804218853
17.4225471463
337.596909208
53.8705095108
37.4852893862
15.3522788832
76.8821189625
142.960019087
24.1265487846
36.5577219758
36.1076781364
17.8702307964
139.391957

21.4509487035
61.0800215527
14.5983279617
6.29279321856
171.028543468
225.427496507
32.0369077535
317.365617433
417.717757195
361.422145627
25.4979087637
37.1483078994
33.1586503494
24.7108963973
47.2145438172
43.4462813342
17.6553210476
62.0046418804
23.8624566194
49.1372204454
48.3493256484
17.1203768332
112.321353971
39.769629735
21.5997486907
38.6233620332
44.062443534
18.9261689243
122.354259043
55.7634545841
36.4722086957
17.0279353196
52.3424349823
19.5293734198
76.0113084451
50.0125678173
30.8765689299
68.1756769936
42.4035424712
19.1706309304
72.2122479899
76.9342969295
83.9633262602
27.182786779
48.6053793471
16.8534610578
78.8640192539
79.0319383467
31.3618615363
78.8328916455
185.629741093
46.9550186956
16.1444193396
7.25269146306
91.9803508772
88.9377438507
18.286591241
427.902876964
480.502233933
479.184736662
439.977341865
31.7411250104
19.3903351224
31.475299609
55.988253197
27.7331358521
89.4918494495
25.6193896748
63.9024925346
51.3271901807
142.006500542
19.871625527

30.1933124573
54.7823883599
17.9868689078
29.8098392347
87.7340975171
22.1199898743
62.3169405402
19.4840130256
210.135470942
33.1717625473
39.1051772845
72.213491271
18.4192558176
45.0733867068
48.9211533078
17.7865681705
57.4121769601
46.7613271495
45.3310853166
21.0406384975
52.4661821548
17.9041000576
66.4675847424
76.7990625114
32.2185231559
54.30011781
48.6454037253
17.5036160668
83.4454878243
79.5415220648
15.8292945266
175.19335032
127.142502046
18.6354705848
91.1013032146
29.2620416364
38.1272634717
41.2301703546
40.9400097609
43.9811253487
16.0931910093
43.7348570952
23.0001316078
95.9948733207
60.2154044936
22.7982280201
64.0
54.584903696
21.6938155901
40.9180430223
52.4215920311
21.6595351335
80.3860704908
87.3503967345
27.9806804536
40.2709885552
52.4405991348
14.8546131316
89.4307889126
128.007812977
29.555043512
165.104078098
18.3210983126
41.4370930933
17.9468479763
71.7981444075
157.007711312
22.9668827755
42.2263790031
50.7637490318
24.3837871776
57.8269453483
43.6093

In [ ]:
netCam.release()